In [1]:
pip install -r requirements.txt

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-igraph

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [21]:
from igraph import Graph
import math
import numpy as np
import pandas as pd
import networkx as nx
import community as louvain_community
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

In [105]:
codes = pd.read_csv('data/codes.csv')
df = pd.read_csv('data/dij_joint_civ.csv')
df['positive_Dij'] = [pow(math.e,d) for d in df['D_ij']]
#codes.set_index('Code', inplace=True)

In [106]:
def build_graph(pos=True, r=5):
    G = nx.Graph()
    G.add_nodes_from(set(df['i']))
    for i, row in df.iterrows():
        if pos:
            G.add_edge(row['i'], row['j'], weight=round(math.e**row['D_ij'], r))
        else:
            G.add_edge(row['i'], row['j'], weight=row['D_ij'])
    return G

G = build_graph(r=2)

In [107]:
codes = codes[(codes['Code'].isin(set(df['i']))) | (codes['Code'].isin(set(df['j'])))]
codes.set_index('Code', inplace=True)

In [110]:
print(G['RU']['UA']['weight'])

0.09


In [43]:
# This is jokes

## Goals: Write a function that takes in a graph and partition method and returns said partition

def country_community_detection(G, detect, verbose=True):
    """
    Performs community detection based on specified algorithm:
    G: Either a weighted igraph Graph or a weighted nx Graph
    detect: 
        - 'fg' == fastgreedy()
        - 'le' == leading_eigenvector()
        - 'ld' == leiden()
        - 'sg' == spinglass()
    vebose: default True, if True will print steps and final groups.
    """
    if isinstance(G, nx.classes.graph.Graph):
        G = Graph.from_networkx(G)
        if verbose:
            print('Converting to network to igraph format.')
    if detect == 'fg':
        groups = G.community_fastgreedy()
        if verbose:
            print('Performed the fastgreedy algorithm')
    elif detect == 'le':
        groups = G.community_leading_eigenvector()
        if verbose:
            print('Performed the leading eigenvector algorithm')
    elif detect == 'ld':
        groups = G.community_leiden()
        if verbose:
            print('Performed the leiden algorithm')
    elif detect == 'ml':
        groups = G.community_multilevel()
        if verbose:
            print('Performed the louvain (multilevel) algorithm')
    elif detect == 'sg':
        groups = G.community_spinglass()
        if verbose:
            print('Performed the spinglass algorithm')
    else:
        raise Exception('Community detection algorithm specified incorrectly.')
    nx_groups = [nx.from_edgelist([(names[x[0]], names[x[1]])
                      for names in [g.vs['_nx_name']] # simply a let
                      for x in g.get_edgelist()], nx.DiGraph()) for g in groups.subgraphs()]
    if verbose:
        for i, g in enumerate(nx_groups):
            countries = []
            for k, v in g.edges():
                countries.append(codes.loc[k]['Name'])
            print('Countries in community ' + str(i) + ':')
            print(set(countries))
    
    return nx_groups

In [44]:
#country_community_detection(G_pos, 'sg')

Converting to network to igraph format.
Performed the spinglass algorithm
Countries in community 0:
{'Ecuador', 'South Africa', 'Canada', 'Albania', 'Madagascar', 'Mexico', 'Mauritania', 'Denmark', 'Norway', 'Malaysia', 'Mauritius', 'Belgium', 'Bangladesh', 'Guatemala', 'Spain', 'Panama', 'Turkey', 'Moldova, Republic of', 'El Salvador', 'Kenya', 'Burundi', 'Macedonia, the Former Yugoslav Republic of', 'Netherlands', 'Mali', 'Indonesia', 'Yemen', "Lao People's Democratic Republic", 'Trinidad and Tobago', 'Malta', 'Bahamas', 'Haiti', 'Pakistan', 'Singapore', 'Bosnia and Herzegovina', 'Guyana', 'Croatia', 'Sudan', 'Bermuda', 'Switzerland', 'Uganda', 'Cayman Islands', 'Mozambique', 'Venezuela, Bolivarian Republic of', 'Nepal', "CÃ´te d'Ivoire", 'Gabon', 'Czech Republic', 'Austria', 'Algeria', 'Cyprus', 'Gambia', 'Peru', 'Hungary', 'Saint Vincent and the Grenadines', 'Iceland', 'Jamaica', 'New Zealand', 'Ireland', 'Korea, Republic of', 'Luxembourg', 'Thailand', 'Belarus', 'Qatar', 'Greece',

In [82]:
weight_partition = louvain_community.best_partition(G_pos)

In [83]:
groups = [[] for num in weight_partition.values()]
for k, v in weight_partition.items():
    groups[v].append(codes.loc[k]['Name'])


In [84]:
for i, g in enumerate(groups):
    print('Countries in groups ' + str(i) + ':')

Countries in groups 0:
Countries in groups 1:
Countries in groups 2:
Countries in groups 3:
Countries in groups 4:
Countries in groups 5:
Countries in groups 6:
Countries in groups 7:
Countries in groups 8:
Countries in groups 9:
Countries in groups 10:
Countries in groups 11:
Countries in groups 12:
Countries in groups 13:
Countries in groups 14:
Countries in groups 15:
Countries in groups 16:
Countries in groups 17:
Countries in groups 18:
Countries in groups 19:
Countries in groups 20:
Countries in groups 21:
Countries in groups 22:
Countries in groups 23:
Countries in groups 24:
Countries in groups 25:
Countries in groups 26:
Countries in groups 27:
Countries in groups 28:
Countries in groups 29:
Countries in groups 30:
Countries in groups 31:
Countries in groups 32:
Countries in groups 33:
Countries in groups 34:
Countries in groups 35:
Countries in groups 36:
Countries in groups 37:
Countries in groups 38:
Countries in groups 39:
Countries in groups 40:
Countries in groups 41:
Co

In [85]:
def __neighcom(node, graph, status, weight_key):
    """
    Compute the communities in the neighborhood of node in the graph given
    with the decomposition node2com
    """
    weights = {}
    for neighbor, datas in graph[node].items():
        if neighbor != node:
            edge_weight = datas.get(weight_key, 1)
            neighborcom = status.node2com[neighbor]
            weights[neighborcom] = weights.get(neighborcom, 0) + edge_weight

    return weights

In [86]:
__neighcom('AM', G_pos, '', 'weight')

AttributeError: 'str' object has no attribute 'node2com'

In [92]:
G = Graph()
G.add_vertices(list(set(df['i'])) + ['ZW'])
for i, row in df.iterrows():
    G.add_edge(row['i'], row['j'], weight=row['positive_Dij'])

In [94]:
from igraph import ARPACKOptions

partition = G.community_multilevel(weights='weight') # arpack_options=ARPACKOptions(max_iter=10000))
groups = {}
for i, g in enumerate(partition.subgraphs()):
    for n in g.vs['name']:
        groups[n] = i
    

In [95]:
codes['ml'] = 0

In [96]:
#codes.set_index('Code', inplace=True)
for k, v in groups.items():
    codes.loc[k, 'ml'] = v

In [97]:
codes.head(10)

,Name,le,inverse_le,ml
Code,,,,
AL,Albania,6.0,1,2
DZ,Algeria,13.0,2,5
AS,American Samoa,0.0,0,5
AO,Angola,2.0,1,1
AR,Argentina,4.0,0,2
AM,Armenia,0.0,0,7
AW,Aruba,7.0,1,9
AU,Australia,15.0,3,5
AT,Austria,20.0,0,11


In [82]:
codes.to_csv('leading_eigenvector_codes.csv')

In [103]:
codes[codes['ml'] == 3]

,Name,le,inverse_le,ml
Code,,,,
KY,Cayman Islands,0.0,0,3
LR,Liberia,0.0,0,3
MD,"Moldova, Republic of",0.0,1,3
PY,Paraguay,0.0,1,3
QA,Qatar,0.0,2,3
SI,Slovenia,0.0,0,3
TN,Tunisia,0.0,0,3
VN,Viet Nam,0.0,1,3
ZM,Zambia,3.0,1,3


KeyError: 'Attribute does not exist'